# Trans-World Oil:  Linear Optimization
### Sean Olson

In [12]:
# import libraries
from optlang import Model, Variable, Constraint, Objective
import pandas as pd

In [13]:
# import data
tankerleasecost = 8600 # lease cost per day
usdistillatecost = 19.80

demand = pd.read_excel('trans_world_oil_company_data.xlsx', sheet_name = 'Demand')
demand['Gasoline'] = demand['Gasoline'] * 1000
demand['Distillate'] = demand['Distillate'] * 1000
display(demand)

refinerycost = pd.read_excel('trans_world_oil_company_data.xlsx', sheet_name = 'RefineryCosts')
display(refinerycost)

variablecost = pd.read_excel('trans_world_oil_company_data.xlsx', sheet_name = 'VariableCosts')
display(variablecost)

tankerusage = pd.read_excel('trans_world_oil_company_data.xlsx', sheet_name = 'TankerUsageFactors')
display(tankerusage)

supply = pd.read_excel('trans_world_oil_company_data.xlsx', sheet_name = 'Supply')
display(supply)

,Area,Gasoline,Distillate
0,Australia,9000.0,21000.0
1,Japan,3000.0,12000.0
2,Philippines,5000.0,8000.0
3,New Zealand,5400.0,8700.0
4,Total,22400.0,49700.0


,"Location, Crude, PI",Country,Cost of Crude,Inc. Ship. Cost,Inc. Ref. Cost,Total Costs,Gasoline Yield per Crude BBL,Distillate Yield per Crude BBL
0,"Brunei Crude, Low (BLA)",Australia,20.5,0.78,0.36,21.64,0.259,0.688
1,"Brunei Crude, High (BHA)",Australia,20.5,0.78,0.84,22.12,0.365,0.573
2,"Iran Crude, Low (ILA)",Australia,18.5,1.86,0.45,20.81,0.186,0.732
3,"Iran Crude, High (IHA)",Australia,18.5,1.86,0.90,21.26,0.312,0.608
4,"Brunei Crude, Low (BLJ)",Japan,20.5,0.72,0.48,21.70,0.259,0.688
5,"Brunei Crude, High (BHJ)",Japan,20.5,0.72,1.02,22.24,0.350,0.588
6,"Iran Crude, Low (ILJ)",Japan,18.5,1.77,0.60,20.87,0.186,0.732
7,"Iran Crude, High (IHJ)",Japan,18.5,1.77,1.17,21.44,0.300,0.620


,From / To,New Zealand,Philippines
0,Australia,0.3,0.45
1,Japan,0.3,0.60
2,US,2.1,1.65


,Between,Australia,Japan,US
0,Iran,0.12,0.110,NaN
1,Borneo,0.05,0.045,NaN
2,Philippines,0.02,0.010,0.15
3,New Zealand,0.01,0.060,0.18


,Supply,BBL/Day,Type
0,Brunei (Borneo),40000,Fixed
1,Iran,60000,Available
2,US Distillate,12000,Available


### Define Objective Function
Now we must identify the objective function for the case.  Given that all the data for the case is cost data then it is safe to assume that the objective function needs to be **Minimize Costs**.

Now we have to define our objective function:

*Total Costs = Cost of Refined Products + Shipment to Other Markets + Tanker Lease*

*Cost of Refined Products = AustraliaCost(Gas + Distillate) + JapanCost(Gas + Distillate) + USCost(Gas + Distillate)*

### Define Constraints
##### Crude Oil Supply:
- Brunei <= 40000
- Iran <= 60000

##### Distillate Supply:
- US <= 12000

##### Refinery Constraint
- Australia <= 50000
- Japan <= 30000

##### Market Demand:  Gas
- Australia >= 9000
- Japan >= 3000
- Philippines >= 5000
- New Zealand >= 5400

##### Market Demand:  Distillate
- Australia >= 21000
- Japan >= 12000
- Philippines >= 8000
- New Zealand >= 8700

##### Tanker Capacity
- Tanker Capacity <= 6900

### Define Decision Variables
##### Quantity and Source of Crude Oil
- Australia
    - High Intensity (Brunei, Gasoline) <= 0.365
    - High Intensity (Brunei, Distillate) <= 0.573
    - Low Intensity (Brunei, Gasoline) <= 0.259
    - Low Intensity (Brunei, Distillate) <= 0.688
    - High Intensity (Iran, Gasoline) <= 0.312
    - High Intensity (Iran, Distillate) <= 0.608
    - Low Intensity (Iran, Gasoline) <= 0.186
    - Low Intensity (Iran, Distillate) <= 0.732
- Japan
    - High Intensity (Brunei, Gasoline) <= 0.350
    - High Intensity (Brunei, Distillate) <= 0.588
    - Low Intensity (Brunei, Gasoline) <= 0.259
    - Low Intensity (Brunei, Distillate) <= 0.688
    - High Intensity (Iran, Gasoline) <= 0.300
    - High Intensity (Iran, Distillate) <= 0.620
    - Low Intensity (Iran, Gasoline) <= 0.186
    - Low Intensity (Iran, Distillate) <= 0.732
    
##### Gas Shipped from Refineries to Other Markets
- Australia -> Philippines >= 0.45
- Australia -> New Zealand >= 0.30
- Japan -> Philippines >= 0.60
- Japan -> New Zealand >= 0.30

##### Distillate Shipped from Refineries to Other Markets
- US -> Philippines >= 1.65
- US -> New Zealand >= 2.10
- Australia -> Philippines >= 0.45
- Australia -> New Zealand >= 0.30
- Japan -> Philippines >= 0.60
- Japan -> New Zealand >= 0.30

##### Number of Tankers Chartered
- Tankers >= 1
- Days Australia -> Philippines >= 2
- Days Australia -> New Zealand >= 1
- Days Japan -> Philippines >= 3
- Days Japan -> New Zealand >= 4
- Days US -> Philippines >= 16
- Days US -> New Zealand >= 14

In [ ]:
# build objective function
# start with Tanker Lease
# tankerlease = 8600 * days * barrels, where barrels <= 6900
